In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import mne_connectivity
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 
import itertools
import sys
import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

# custom scripts
sys.path.append('/home/brooke/pacman/connectivity/scripts/turnaround_coherence')
from connectivity_functions import *

In [2]:
subject = 'BJH016'

In [3]:
preproc_data_dir = f"/home/brooke/pacman/preprocessing/"

## Load Neural Data

# load
last_away_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}/ieeg/{subject}_bp_filtered_clean_last_away_events.fif")

# get good epochs (for behavioral data only)
good_epochs = [i for i,x in enumerate(last_away_epochs.get_annotations_per_epoch()) if not x]
bad_epochs = [i for i,x in enumerate(last_away_epochs.get_annotations_per_epoch()) if  x]

# load behavioral data
last_away_data = pd.read_csv(f"{preproc_data_dir}/../raw_data/{subject}/behave/{subject}_last_away_events.csv")

# set info as metadata
last_away_epochs.metadata = last_away_data

# onlt good epochs
last_away_epochs = last_away_epochs[good_epochs]


## Dictionary of electrode locations ##

# Dictionary mapping ROI to elecs
# Pull mapping ROI to elecs
with open('/home/brooke/pacman/preprocessing/scripts/roi.py') as file:
    exec(file.read())
ROIs = ROIs[subject]

## prep lists

# primary ROI
hc_list = []
hc_indices = []
hc_names = []
ofc_list = []
ofc_indices = []
ofc_names = []
amyg_list = []
amyg_names = [] 
amyg_indices = []
cing_list = []
cing_names = [] 
cing_indices = []

# control ROI
insula_list = []
insula_names = []  
insula_indices = []
dlpfc_list = []
dlpfc_names = []  
dlpfc_indices = []
ec_list = []
ec_names = []  
ec_indices = []

# exclude bad ROI from list
pairs_long_name = [ch.split('-') for ch in last_away_epochs.info['ch_names']]
bidx = len(last_away_epochs.info['bads']) +1
pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

# sort ROI into lists
for ix in range(0, len(pairs_name)):
    if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
        hc_list.append(last_away_epochs.info['ch_names'][ix + bidx])
        hc_names.append(pairs_name[ix])
        hc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
        ofc_list.append(last_away_epochs.info['ch_names'][ix + bidx])
        ofc_names.append(pairs_name[ix])
        ofc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
        amyg_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        amyg_names.append(pairs_name[ix])
        amyg_indices.append(ix)
    if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
        cing_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        cing_names.append(pairs_name[ix])
        cing_indices.append(ix)
        
    # control roi
    if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
        insula_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        insula_names.append(pairs_name[ix])
        insula_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
        dlpfc_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        dlpfc_names.append(pairs_name[ix])
        dlpfc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
        ec_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        ec_names.append(pairs_name[ix])
        ec_indices.append(ix)        
        

## Set frequencies ##
freqs = np.logspace(start = np.log10(1), stop = np.log10(150), num = 80, base = 10, endpoint = True)
n_cycles = np.logspace(np.log10(2), np.log10(30), base = 10, num = 80)

# delta, theta, hfa
delta_freqs = freqs[np.where((freqs <= 3))]
delta_cycles = n_cycles[np.where((freqs <= 3))]

theta_freqs = freqs[np.where((freqs > 3) & (freqs < 8))]
theta_cycles = n_cycles[np.where((freqs > 3) & (freqs < 8))]

hfa_freqs = freqs[np.where((freqs > 70))]
hfa_cycles = n_cycles[np.where((freqs > 70))]

# resample 
if last_away_epochs.info['sfreq'] > 1000:
    last_away_epochs= last_away_epochs.resample(100)

# Crop #
last_away_epochs.crop(tmin = -2, tmax = 1) 

## remove any electrodes that are duplicated across regions... ugh so much code for such a simple thing ##
# Combine all ROI lists into a single Series
elec_list = pd.Series(hc_list + amyg_list + ofc_list + cing_list + dlpfc_list + insula_list)

# Identify duplicated elements
duplicated_list = elec_list[elec_list.duplicated()].tolist()

# ROI lists
roi_lists = [hc_list, amyg_list, ofc_list, cing_list, dlpfc_list, insula_list]

# Find indices of last occurrences of duplicated elements
items_to_remove = [(sub_roi, idx_list) for idx_list, sub_list in enumerate(roi_lists) 
                for idx, sub_roi in enumerate(sub_list) 
                if sub_roi in duplicated_list and 
                    idx == len(sub_list) - sub_list[::-1].index(sub_roi) - 1]

# Convert to list of tuples with unique first elements
items_to_remove = list({t[0]: t for t in items_to_remove}.values())

# remove elements
for item in items_to_remove:
    roi_lists[item[1]].remove(item[0])

# reset roi lists
hc_list = roi_lists[0]
amyg_list = roi_lists[1]
ofc_list = roi_lists[2]
cing_list = roi_lists[3]
dlpfc_list = roi_lists[4]
insula_list = roi_lists[5]
roi_lists = [hc_list, amyg_list, ofc_list, cing_list, dlpfc_list, insula_list]

# only ROI of interest
last_away_hc = last_away_epochs.copy().pick_channels(hc_list)
last_away_amyg = last_away_epochs.copy().pick_channels(amyg_list)
last_away_ofc = last_away_epochs.copy().pick_channels(ofc_list)
last_away_cing = last_away_epochs.copy().pick_channels(cing_list)
last_away_dlpfc = last_away_epochs.copy().pick_channels(dlpfc_list)
last_away_insula = last_away_epochs.copy().pick_channels(insula_list)

## combine ##
last_away_roi = last_away_hc.add_channels([last_away_amyg, last_away_ofc, last_away_cing, last_away_dlpfc, last_away_insula])

## get indicies for all the noon-symmetric pairs ##
first_pair_indices, second_pair_indices = get_indices_of_connectivity_pairs(roi_lists, last_away_roi.info['ch_names'])


Reading /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_last_away_events.fif ...


/tmp/ipykernel_2190/1531861651.py:6: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing//BJH016/ieeg/BJH016_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}/ieeg/{subject}_bp_filtered_clean_last_away_events.fif")


    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Reading /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_last_away_events-1.fif ...
    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Reading /home/brooke/pacman/preprocessing/BJH016/ieeg/BJH016_bp_filtered_clean_last_away_events-2.fif ...
    Found the data of interest:
        t =   -5000.00 ...    5000.00 ms
        0 CTF compensation matrices available
Not setting metadata
304 matching events found
No baseline correction applied
0 projection items activated
Adding metadata with 5 columns


In [4]:

sys.getsizeof(last_away_roi._data)

29401824

In [5]:
roi_coherence = compute_epoch_coherence(last_away_roi, last_away_roi.info.ch_names, (first_pair_indices, second_pair_indices), theta_freqs, theta_cycles, workers = 16)


computing TFR
Fmin was not specified. Using fmin=min(freqs)
Fmax was not specified. Using fmax=max(freqs).


: 

In [20]:
pair_names = [f"{last_away_roi.info['ch_names'][x]}_{last_away_roi.info['ch_names'][y]}" for x, y in zip(first_pair_indices, second_pair_indices)]

In [21]:
coherence_df = pd.DataFrame(roi_coherence[0].get_data().mean(axis = 2), columns=pair_names)
coherence_df['neural_trial_numeric'] = last_away_roi.metadata['neural_trial_numeric']

,LH2-LH3_LA1-LA2,LH2-LH3_LA2-LA3,LH2-LH3_LA3-LA4,RH1-RH2_LA1-LA2,RH1-RH2_LA2-LA3,RH1-RH2_LA3-LA4,LH2-LH3_ROF1-ROF2,RH1-RH2_ROF1-ROF2,LH2-LH3_LAC1-LAC2,RH1-RH2_LAC1-LAC2,...,LMC5-LMC6_LAI3-LAI4,LMC6-LMC7_LAI1-LAI2,LMC6-LMC7_LAI2-LAI3,LMC6-LMC7_LAI3-LAI4,LMC7-LMC8_LAI1-LAI2,LMC7-LMC8_LAI2-LAI3,LMC7-LMC8_LAI3-LAI4,ROF9-ROF10_LAI1-LAI2,ROF9-ROF10_LAI2-LAI3,ROF9-ROF10_LAI3-LAI4
0,0.317028,0.323996,0.290633,0.161074,0.368216,0.313641,0.219614,0.292650,0.342380,0.234980,...,0.300531,0.263019,0.293437,0.248708,0.550967,0.597414,0.275447,0.356820,0.187874,0.196450
1,0.581392,0.318964,0.335771,0.329440,0.256912,0.522040,0.259349,0.365444,0.459440,0.145418,...,0.443441,0.444177,0.395800,0.398141,0.585543,0.463335,0.367086,0.280865,0.205835,0.160568
2,0.673273,0.552537,0.411159,0.270099,0.264566,0.402408,0.453135,0.185926,0.460036,0.473869,...,0.339118,0.411578,0.410802,0.455480,0.540952,0.427822,0.292165,0.255619,0.261745,0.141792
3,0.335646,0.210468,0.482617,0.183828,0.404902,0.290365,0.281925,0.309370,0.346901,0.387549,...,0.290688,0.308860,0.309487,0.364565,0.431389,0.373341,0.315394,0.322024,0.385375,0.425612
4,0.391680,0.626676,0.473404,0.376078,0.244988,0.182818,0.333379,0.244429,0.293454,0.433082,...,0.333371,0.334446,0.370631,0.423891,0.399708,0.392517,0.379408,0.407312,0.370530,0.425325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0.252975,0.369640,0.380394,0.253482,0.539759,0.565688,0.316096,0.268402,0.353051,0.345162,...,0.282232,0.408466,0.269174,0.212582,0.396375,0.469210,0.563274,0.310790,0.196085,0.126507
155,0.421666,0.417955,0.399676,0.394521,0.327040,0.287986,0.533202,0.357678,0.490599,0.568360,...,0.153639,0.336532,0.302631,0.254717,0.175084,0.211019,0.253068,0.535131,0.519815,0.498767
156,0.408193,0.602335,0.491497,0.509237,0.418598,0.489588,0.506322,0.448068,0.628371,0.269441,...,0.388045,0.334360,0.410920,0.434055,0.322689,0.350654,0.304035,0.282678,0.292662,0.288895
157,0.255023,0.416791,0.433105,0.351554,0.354715,0.275655,0.324694,0.228547,0.284332,0.322926,...,0.388667,0.316544,0.396811,0.314625,0.347957,0.415116,0.437258,0.443406,0.516367,0.339946


In [25]:
coherence_df = pd.DataFrame(roi_coherence[0].get_data().mean(axis = 2), columns=pair_names)
iplv_df = pd.DataFrame(roi_coherence[1].get_data().mean(axis = 2), columns=pair_names)
pli_df = pd.DataFrame(roi_coherence[2].get_data().mean(axis = 2), columns=pair_names)

coherence_df['neural_trial_numeric'] = last_away_roi.metadata['neural_trial_numeric'].reset_index(drop=True)
iplv_df['neural_trial_numeric'] = last_away_roi.metadata['neural_trial_numeric'].reset_index(drop=True)
pli_df['neural_trial_numeric'] = last_away_roi.metadata['neural_trial_numeric'].reset_index(drop=True)

# save
coherence_df.to_csv(f"/home/brooke/pacman/connectivity/ieeg/{subject}_theta_coherence.csv")
iplv_df.to_csv(f"/home/brooke/pacman/connectivity/ieeg/{subject}_theta_iplv.csv")
pli_df.to_csv(f"/home/brooke/pacman/connectivity/ieeg/{subject}_theta_pli.csv")

In [30]:
tmp = last_away_roi.metadata['neural_trial_numeric']


In [32]:
type(tmp)

pandas.core.series.Series